<a href="https://colab.research.google.com/github/cesaroliveiragoes/Statistics-for-Data-Science/blob/main/checkpoint_02_2_sem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checkpoint 2 - Análise Covid Brasil

## Carregar módulos

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#datas
import datetime

# estatistica
from scipy.stats import binom, poisson
from scipy.stats import expon, norm, t, chi2, f

## Carregar dados

- Dados Covid Brasil

In [ ]:
! wget --no-check-certificate --content-disposition 'https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true'

--2023-11-14 21:37:06--  https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/wcota/covid19br/raw/master/cases-brazil-cities-time.csv.gz [following]
--2023-11-14 21:37:07--  https://github.com/wcota/covid19br/raw/master/cases-brazil-cities-time.csv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv.gz [following]
--2023-11-14 21:37:07--  https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|18

In [ ]:
! gunzip cases-brazil-cities-time.csv.gz

In [ ]:
dados_brasil = pd.read_csv('cases-brazil-cities-time.csv')

- Dados censo

In [ ]:
dados_censo = pd.read_csv('dados_municipios_2010.csv', sep = ';', decimal = ',', encoding = 'latin1')

FileNotFoundError: ignored

## Ajustar dados

In [ ]:
max_date = max(dados_brasil['date'])

In [ ]:
dados_brasil = dados_brasil[(dados_brasil['date'] == max_date) & \
                            (dados_brasil['state'] != 'TOTAL')]

In [ ]:
cols = ['ibgeID', 'date', 'state', 'city', 'totalCases', 'deaths']

In [ ]:
dados_brasil = dados_brasil[cols].reset_index(drop=True)

## Aqui começa o trabalho

1 –Realizar o join com o dataframe “dados_brasil”com o dataframe “dados_censo”.

In [ ]:
# exibir/visualizar a tabela dados_brasil
dados_brasil

In [ ]:
# exibir/visualizar a tabela dados_censo
dados_censo

In [ ]:
#realizar o join usando a função pd.merge pela coluna ibgeID que é a coluna em comum das duas tabelas
dados_merge = pd.merge(dados_brasil, dados_censo, on=['ibgeID'], how = 'left')
dados_merge

In [ ]:
# visualizar dados ausentes e/ou duplicatas
dados_sem_merge = dados_merge[dados_merge.isnull().any(axis=1) == True]

In [ ]:
dados_sem_merge[['ibgeID', 'state', 'city']].drop_duplicates()

In [ ]:
# dados sem duplicatas ou valores ausentes
dados_merge_final = dados_merge[~dados_merge.isnull().any(axis=1) == True].reset_index()
dados_merge_final

2 –Qual  seria  o  tamanho  de  amostra  para  uma  pesquisa  em  todos  os  municípios  do  Brasil contidos  no  dataframe?  (Utilizar  a  variância  da  renda  per  capita  como  base  da  fórmula  de amostragem, z = 1,96 e margem de erro = 20).

In [ ]:
# descobrir o número total de municípios no dataframe
len(dados_merge_final['ibgeID'].unique())

In [ ]:
# forma alternativa de como descobrir o número total de municípios no dataframe
dados_merge_final.shape

In [ ]:
# descorbrir a variância (ou seja o desvio padrão) da renda per capta
variancia_rdpc = dados_merge_final.agg(dp_RDPC = pd.NamedAgg('RDPC', 'std'))
variancia_rdpc

In [ ]:
# para este exercício usaremos amostra contínua
def formula_amostra_continua(N, S, Z, ME):
  n = (Z**2 * S**2 * N) / ((ME**2 * (N-1)) + (Z**2 * S**2))
  return int(n)

In [ ]:
# nomear as variáveis para a fórmula de AMOSTRA CONTÍNUA
# S = variância/desvio padrão   N = população total   Z = valor da distribuição para o nível de confiança adotado  ME = margem de erro
N = 5565
Z = 1.96
S = 243.269333
ME = 20

In [ ]:
n_br = formula_amostra_continua(N, S, Z, ME)
n_br

3 –Qual seria o tamanho da amostra para São Paulo? (Utilizar a variância da renda per capita como base da fórmula de amostragem, z = 1,96 e margem de erro = 20)

In [ ]:
estado = 'SP'

In [ ]:
dados_sp = dados_merge_final[dados_merge_final['state'] == estado].reset_index()
dados_sp

In [ ]:
# descorbrir a variância (ou seja o desvio padrão) da renda per capta
variancia_rdpc_sp = dados_sp.agg(dp_RDPC = pd.NamedAgg('RDPC', 'std'))
variancia_rdpc_sp

In [ ]:
# neste exercício usaremos amostra contínua
def formula_amostra_continua(N, S, Z, ME):
  n = (Z**2 * S**2 * N) / ((ME**2 * (N-1)) + (Z**2 * S**2))
  return int(n)

In [ ]:
# nomear as variáveis para a fórmula de AMOSTRA CONTÍNUA
# S = variância/desvio padrão   N = população total   Z = valor da distribuição para o nível de confiança adotado  ME = margem de erro
N = 645
Z = 1.96
S = 197.39827
ME = 25

In [ ]:
n_sp = formula_amostra_continua(N, S, Z, ME)
n_sp

4 –Agora que possui os tamanhos de amostra para o Brasil (n_br) e para São Paulo (n_sp), criar um dataframe para a amostra do Brasil e outro dataframe para amostra de São Paulo. (utilizar o método da amostra aleatória simples)

In [ ]:
# dataframe para Brasil
dados_amostra_simples_br = dados_merge_final.sample(n=n_br, random_state=1245)
dados_amostra_simples_br

In [ ]:
# dataframe para SP
dados_amostra_simples_sp = dados_sp.sample(n=n_sp, random_state=1245)
dados_amostra_simples_sp

5 –Agora    que    possui    os    dataframes    de    Brasil    (dados_amostra_br)    e    São    Paulo (dados_amostra_sp), realizar as seguintes análises:

a) Obter as medidas descritivas: média, desvio padrão, primeiro, segundo e terceiro quartil para o número de infectados.

Brasil

In [ ]:
# Criar a função do primeiro quartil
def q1_quantile(x):
  return np.quantile(x, 0.25)

In [ ]:
# Criar a função do segundo quartil
def q2_quantile(x):
  return np.quantile(x, 0.50)

In [ ]:
# Criar a função do terceiro quartil
def q3_quantile(x):
  return np.quantile(x, 0.75)

In [ ]:
# medidas descritivas Brasil para o número de infectados, note que, temos que realizar o groupby por uma coluna que todos os dados são
# iguais para todos os registros, no caso deste dataframe é a coluna "date"
dados_descritivos_br = dados_amostra_simples_br.groupby('date').agg(media_infectados = pd.NamedAgg('totalCases','mean'),
                          dp_infectados = pd.NamedAgg('totalCases','std'),
                          q1_infectados = pd.NamedAgg('totalCases', q1_quantile),
                          q2_infectados = pd.NamedAgg('totalCases', q2_quantile),
                          q3_infectados = pd.NamedAgg('totalCases', q3_quantile)).reset_index()

In [ ]:
dados_descritivos_br

Estado de São Paulo

In [ ]:
# Criar a função do primeiro quartil
def q1_quantile(x):
  return np.quantile(x, 0.25)

In [ ]:
# Criar a função do segundo quartil
def q2_quantile(x):
  return np.quantile(x, 0.50)

In [ ]:
# Criar a função do terceiro quartil
def q3_quantile(x):
  return np.quantile(x, 0.75)

In [ ]:
# medidas descritivas São Paulo para o número de infectados, note que temos que realizar o groupby por uma coluna que todos os dados são
# iguais para todos os registros, no caso deste dataframe é a coluna "state"
dados_descritivos_sp = dados_amostra_simples_sp.groupby('state').agg(media_infectados = pd.NamedAgg('totalCases','mean'),
                          dp_infectados = pd.NamedAgg('totalCases','std'),
                          q1_infectados = pd.NamedAgg('totalCases', q1_quantile),
                          q2_infectados = pd.NamedAgg('totalCases', q2_quantile),
                          q3_infectados = pd.NamedAgg('totalCases', q3_quantile)).reset_index()

In [ ]:
dados_descritivos_sp

b) Obter as medidas descritivas: média, desvio padrão, primeiro, segundo e terceiro quartil para o número de mortes.

In [ ]:
# Criar a função do primeiro quartil
def q1_quantile(x):
  return np.quantile(x, 0.25)

In [ ]:
# Criar a função do segundo quartil
def q2_quantile(x):
  return np.quantile(x, 0.50)

In [ ]:
# Criar a função do terceiro quartil
def q3_quantile(x):
  return np.quantile(x, 0.75)

In [ ]:
# medidas descritivas Brasil para o número de infectados opção
dados_descritivos_br_mortes = dados_amostra_simples_br.groupby('date').agg(media_mortos = pd.NamedAgg('deaths','mean'),
                          dp_mortos = pd.NamedAgg('deaths','std'),
                          q1_mortos = pd.NamedAgg('deaths', q1_quantile),
                          q2_mortos = pd.NamedAgg('deaths', q2_quantile),
                          q3_mortos = pd.NamedAgg('deaths', q3_quantile)).reset_index()

In [ ]:
dados_descritivos_br_mortes

Estado de São Paulo

In [ ]:
# Criar a função do primeiro quartil
def q1_quantile(x):
  return np.quantile(x, 0.25)

In [ ]:
# Criar a função do segundo quartil
def q2_quantile(x):
  return np.quantile(x, 0.50)

In [ ]:
# Criar a função do terceiro quartil
def q3_quantile(x):
  return np.quantile(x, 0.75)

In [ ]:
# medidas descritivas SP para o número de infectados opção 1
dados_descritivos_sp_mortes = dados_amostra_simples_sp.groupby('state').agg(media_mortos = pd.NamedAgg('deaths','mean'),
                          dp_mortos = pd.NamedAgg('deaths','std'),
                          q1_mortos = pd.NamedAgg('deaths', q1_quantile),
                          q2_mortos = pd.NamedAgg('deaths', q2_quantile),
                          q3_mortos = pd.NamedAgg('deaths', q3_quantile)).reset_index()

In [ ]:
dados_descritivos_sp_mortes

c) Gerar um histograma par o número de infectados e mortes. O desenho do histograma é similar com alguma das distribuições apresentadas?

In [ ]:
# a distribuição de Poisson cabe neste exercício pois
# consite de n eventos que podem ser contados (infectados e mortos)
# eventos independentes
# obter a média dos eventos dentro de um intervalo

# Infectados Brasil

plt.hist(dados_amostra_simples_br['totalCases'])
plt.xlabel('Número de Infectados')
plt.ylabel('Frequência')

In [ ]:
# Infectados São Paulo
plt.hist(dados_amostra_simples_sp['totalCases'])
plt.xlabel('Número de Infectados')
plt.ylabel('Frequência')

In [ ]:
# Mortes Brasil
plt.hist(dados_amostra_simples_br['deaths'])
plt.xlabel('Número de Mortos')
plt.ylabel('Frequência')

In [ ]:
# Mortes SP
plt.hist(dados_amostra_simples_sp['deaths'])
plt.xlabel('Número de Mortos')
plt.ylabel('Frequência')

In [ ]:
# O desenho do histograma é similar com alguma das distribuições de Poisson, uma vez que, a barra maior
# condiz com o apresentado no exercício anterior das médias de infectados e mortos, ou seja, o que o gráfico nos diz
# é que a probabilidade do número de pessoas serem infectadas e/ou morrerem condiz com a altura da barra, então
# quanto maior a barra mais próximo da média está, ou seja, mais probabilidades da ocorrência.

d) Compare os resultados de Brasil e SP.

In [ ]:
# São Paulo demonstrou uma alta frequência de muitos casos de infecção e mortes, principalmente por causa de municípios com
# grande densidade populacional como a região metropolitana, o que fez com que esses números aumentassem.

# Em comparação ao Brasil, a média sem mantém muito parecida, o que de fato os difere é como mencionado, as altas frequencias com muitos
# infectados e mortos.

In [ ]:
# INTEGRANTES
# Cesar Oliveira Goes - RM: 98119
# Fiama dos Santos Trajano - RM: 97885
# Karina Maciel Palmeira - RM: 551770